In [1]:
from matplotlib import pyplot as plt
from sklearn import datasets
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.pipeline.steps import AutoMLStep
from azureml.widgets import RunDetails
from azureml.core.environment import Environment 
from azureml.core.model import InferenceConfig 
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import Model
import pandas as pd

In [2]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="hyperdrive")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-226820
Azure region: eastus2
Subscription id: d7f39349-a66b-446e-aba6-0053c2cf1c11
Resource group: aml-quickstarts-226820


In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute

cluster_name = "Proj1-test2" 

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)
compute_target.wait_for_completion(show_output=True)
print(compute_target.get_status().serialize())

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 1, 'targetNodeCount': 1, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 1, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2023-02-26T19:59:57.932000+00:00', 'errors': None, 'creationTime': '2023-02-26T19:44:22.605314+00:00', 'modifiedTime': '2023-02-26T19:44:26.038620+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT1800S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


In [4]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler -- Randomly sampling C and Max_Iter values  
ps = RandomParameterSampling(
    {
        '--C' : choice(0.001,0.01,0.1,1,10,20,50,100,200,500,1000),
        '--max_iter': choice(50,100,300)
    }
)

# Specify a Policy -- Bandit policy for early termination
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory='.',
                      script='train.py',
                      compute_target=compute_target,
                      environment=sklearn_env
                      )

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling=ps,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     policy=policy,
                                     run_config=src,
                                     max_concurrent_runs=4,
                                     max_total_runs=16,                                     
                                    )

In [5]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###

hyperdrive_run = exp.submit(hyperdrive_config) #use the experiment to submit the hyperdrive config

# Monitor HyperDrive runs 
# You can monitor the progress of the runs with the following Jupyter widget
RunDetails(hyperdrive_run).show()

hyperdrive_run.wait_for_completion(show_output=True)

In [6]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###

#Best Run and metrics
best_run = hyperdrive_run.get_best_run_by_primary_metric()
print("best run details :",best_run.get_details())
print("best run file names :",best_run.get_file_names())
print("best run metrics :",best_run.get_metrics())

#save the model
#best_run.register_model(model_name = "hyperdrive_best_run.pkl", model_path = './outputs/')
best_run.register_model(model_name = 'hyperdrive_best_run.pkl', model_path = 'outputs/')

print(best_run)
#best_run.download_file( name='hyperdrive_best_run.pkl')



best run details : {'runId': 'HD_887a2b34-8ac1-4850-aec3-53c04a5ca594_5', 'target': 'Proj1-test2', 'status': 'Completed', 'startTimeUtc': '2023-02-26T20:56:34.366471Z', 'endTimeUtc': '2023-02-26T20:57:00.852848Z', 'services': {}, 'properties': {'_azureml.ComputeTargetType': 'amlctrain', 'ContentSnapshotId': 'b84e160d-81ed-4047-ab97-5fe442af938b', 'ProcessInfoFile': 'azureml-logs/process_info.json', 'ProcessStatusFile': 'azureml-logs/process_status.json'}, 'inputDatasets': [], 'outputDatasets': [], 'runDefinition': {'script': 'train.py', 'command': '', 'useAbsolutePath': False, 'arguments': ['--C', '200', '--max_iter', '300'], 'sourceDirectoryDataStore': None, 'framework': 'Python', 'communicator': 'None', 'target': 'Proj1-test2', 'dataReferences': {}, 'data': {}, 'outputData': {}, 'datacaches': [], 'jobName': None, 'maxRunDurationSeconds': 2592000, 'nodeCount': 1, 'instanceTypes': [], 'priority': None, 'credentialPassthrough': False, 'identity': None, 'environment': {'name': 'sklearn-e

In [7]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###

#df = TabularDatasetFactory.from_delimited_files(path='https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv')


data = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'
dataset = Dataset.Tabular.from_delimited_files(data)

In [8]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(dataset)

In [9]:
# #Merge the cleaned dataset into one dataset

clean_df_merged = pd.concat([x, y], axis=1)
clean_df_merged = clean_df_merged.rename(columns= {'y':'Label_col'})
clean_df_merged.head()

,age,marital,default,housing,loan,month,day_of_week,duration,campaign,pdays,...,contact_telephone,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,Label_col
0,57,1,0,0,1,5,1,371,1,999,...,0,0,0,0,1,0,0,0,0,0
1,55,1,0,1,0,5,4,285,2,999,...,1,0,0,0,0,0,0,0,1,0
2,33,1,0,0,0,5,5,52,1,999,...,0,0,0,1,0,0,0,0,0,0
3,36,1,0,0,0,6,5,355,4,999,...,1,0,0,0,1,0,0,0,0,0
4,27,1,0,1,0,7,5,189,2,999,...,0,0,0,0,1,0,0,0,0,0


In [10]:
# The default datastore is a blob storage container where datasets are stored
datastore = ws.get_default_datastore()

In [11]:
#Registering dataset into a Azure Tabular Dataset 
Main_dataset = Dataset.Tabular.register_pandas_dataframe(
        dataframe=clean_df_merged, 
        name='clean_df_merged_data', 
        target=datastore
    )

Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/d83ec0ed-0bda-4160-885a-c703ff39a93a/
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'emp.var.rate' -> 'emp_var_rate'
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'cons.price.idx' -> 'cons_price_idx'
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'cons.conf.idx' -> 'cons_conf_idx'
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'nr.employed' -> 'nr_employed'
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'job_admin.' -> 'job_admin_'
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'education_basic.4y'

In [12]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    compute_target = compute_target,
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric= 'accuracy',
    training_data=Main_dataset,
    label_column_name='Label_col', 
    n_cross_validations= 2)

In [13]:
# Submit your automl run

### YOUR CODE HERE ###

ws = Workspace.from_config()
#exp = Experiment(workspace=ws, name="hyperdrive")

experiment = Experiment(ws, "AutoML")

remote_run = experiment.submit(automl_config, show_output= True)
remote_run.wait_for_completion()

Submitting remote run.
No run_configuration provided, running on Proj1-test2 with default configuration
Running on remote compute: Proj1-test2


Experiment,Id,Type,Status,Details Page,Docs Page
AutoML,AutoML_c36df95d-a166-416b-9a5b-96d555378323,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+-------------------------------------

{'runId': 'AutoML_c36df95d-a166-416b-9a5b-96d555378323',
 'target': 'Proj1-test2',
 'status': 'Completed',
 'startTimeUtc': '2023-02-26T21:02:23.329537Z',
 'endTimeUtc': '2023-02-26T21:29:23.203673Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 10 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '2',
  'target': 'Proj1-test2',
  'AMLSettingsJsonString': '{"path":null,"name":"AutoML","subscription_id":"d7f39349-a66b-446e-aba6-0053c2cf1c11","resource_group":"aml-quickstarts-226820","workspace_name":"quick-starts-ws-226820","region":"eastus2","compute_target":"Proj1-test2","spark_service":n

In [15]:
%pip install xgboost==1.3.3

     ---------------------------------------- 95.2/95.2 MB 7.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [16]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###

best_run, fitted_model = remote_run.get_output()

print("best run details :",best_run.get_details())
print("best run file names :",best_run.get_file_names())
print("best run metrics :",best_run.get_metrics())

#save the model
best_run.register_model(model_name = "AutoML_best_run.pkl", model_path = 'outputs/')

print(best_run)
#best_run.download_file( name= './outputs/AutoML_best_run.pkl')



best run details : {'runId': 'AutoML_c36df95d-a166-416b-9a5b-96d555378323_30', 'target': 'Proj1-test2', 'status': 'Completed', 'startTimeUtc': '2023-02-26T21:26:45.229952Z', 'endTimeUtc': '2023-02-26T21:27:53.913095Z', 'services': {}, 'properties': {'runTemplate': 'automl_child', 'pipeline_id': '__AutoML_Ensemble__', 'pipeline_spec': '{"pipeline_id":"__AutoML_Ensemble__","objects":[{"module":"azureml.train.automl.ensemble","class_name":"Ensemble","spec_class":"sklearn","param_args":[],"param_kwargs":{"automl_settings":"{\'task_type\':\'classification\',\'primary_metric\':\'accuracy\',\'verbosity\':20,\'ensemble_iterations\':15,\'is_timeseries\':False,\'name\':\'AutoML\',\'compute_target\':\'Proj1-test2\',\'subscription_id\':\'d7f39349-a66b-446e-aba6-0053c2cf1c11\',\'region\':\'eastus2\',\'spark_service\':None}","ensemble_run_id":"AutoML_c36df95d-a166-416b-9a5b-96d555378323_30","experiment_name":"AutoML","workspace_name":"quick-starts-ws-226820","subscription_id":"d7f39349-a66b-446e-aba

In [17]:
#compute cluster delete after use

compute_target.delete()